In [1]:
%load_ext autoreload
%autoreload 2

from transformer_lens import HookedTransformer
import torch as t
from huggingface_hub import login
from contextlib import contextmanager
import json
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc
from IPython.display import display, HTML
import html as html_escaper
import copy

from utils import *

# login("")
t.set_grad_enabled(False)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model_base = HookedTransformer.from_pretrained_no_processing("gemma-2-2b", device="cpu")
model_chat = HookedTransformer.from_pretrained_no_processing("gemma-2-2b-it", device="cuda")
model_hs = copy.deepcopy(model_chat)
clear_mem()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [32]:
MAX_SEQ_LEN = 300
NUM_LINES = 2000
with open("2b_it_generations_deduped.jsonl", "r") as f:
    data = [json.loads(line)['conversation'] for i, line in enumerate(f) if i < NUM_LINES]
toks = model_base.to_tokens(data, prepend_bos=False)[:, :MAX_SEQ_LEN] 

In [33]:
clear_mem()

layer, head = 6, 0
kl_hs, kl_ablated = get_kl(model_base, model_hs, model_chat, toks[:200], layer, head, batch_size=16)

  0%|          | 0/12 [00:00<?, ?it/s]

In [35]:
top_vals_hs, top_inds_hs = matrix_topk(kl_hs, k=20)
top_vals_ablated, top_inds_ablated = matrix_topk(kl_ablated, k=20)

already_printed = []
for (batch_idx, seq_idx) in top_inds_hs:
    # if batch_idx in already_printed:
    #     continue
    # already_printed.append(batch_idx)
    highlight_kl(model_base, model_hs, model_chat, toks[batch_idx, :seq_idx+10], layer, head, cscale=0.1, ablate=False)
    print("--"*100)

High KL: 0.1104 at token ' following'\n


,Token,p_HS,p_B
0,group,0.035,0.163
1,name,0.034,0.044
2,music,0.071,0.083
3,high,0.030,0.024
4,school,0.723,0.547


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0469 at token ' water'\n


,Token,p_HS,p_B
0,hide,0.070,0.120
1,hay,0.086,0.146
2,wheel,0.028,0.036
3,water,0.645,0.508


...

,Token,p_HS,p_B
0,"""",0.125,0.150
1,“,0.019,0.023
2,a,0.015,0.018
3,**,0.480,0.436


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0069 at token '<start_of_turn>'\n


,Token,p_HS,p_B
0,\,0.007,0.010
1,,0.080,0.094
2,{,0.504,0.531
3,,0.094,0.086
4,*,0.283,0.239


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [16]:
top_vals_hs, top_inds_hs = matrix_topk(kl_hs, k=10)
top_vals_ablated, top_inds_ablated = matrix_topk(kl_ablated, k=10)
for (batch_idx, seq_idx) in top_inds_hs:
    highlight_kl(model_base, model_hs, model_chat, toks[batch_idx, :seq_idx+30], layer, head, cscale=0.1, ablate=False)
    print("--"*100)

High KL: 0.0491 at token 'Fighter'\n


,Token,p_HS,p_B
0,Stri,0.025,0.036
1,Fighter,0.621,0.719
2,奋斗,0.232,0.123


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0427 at token ' src'\n


,Token,p_HS,p_B
0,window,0.377,0.488
1,*,0.103,0.120
2,COM,0.393,0.264


...

,Token,p_HS,p_B
0,#,0.010,0.015
1,[,0.318,0.402
2,",",0.212,0.230
3,.,0.393,0.279


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0327 at token ' crews'\n


,Token,p_HS,p_B
0,dogs,0.012,0.020
1,devils,0.014,0.021
2,crews,0.079,0.100
3,rivals,0.158,0.145
4,ambition,0.012,0.007
5,swords,0.013,0.007


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
